In [1]:
from collections import Counter
import pandas as pd

In [2]:
df_counties = pd.read_csv('./migration_v2/origin/CountyName_MSAName.csv', dtype={'fipscounty': str})
df_counties = df_counties.drop_duplicates()
df_counties.shape

(3292, 7)

In [3]:
df_counties

,countyname,state,fipscounty,msa,msaname,cbsa,cbsaname
0,AUTAUGA,AL,01001,5240.0,"MONTGOMERY, AL",33860.0,"Montgomery, AL"
1,BALDWIN,AL,01003,5160.0,"MOBILE, AL",NaN,NaN
2,BARBOUR,AL,01005,1.0,ALABAMA,NaN,NaN
3,BIBB,AL,01007,1.0,ALABAMA,13820.0,"Birmingham-Hoover, AL"
4,BLOUNT,AL,01009,1000.0,"BIRMINGHAM, AL",13820.0,"Birmingham-Hoover, AL"
...,...,...,...,...,...,...,...
3288,VIEQUES,PR,72147,40.0,PUERTO RICO,NaN,NaN
3289,VILLALBA,PR,72149,6360.0,"PONCE, PR",38660.0,"Ponce, PR"
3290,YABUCOA,PR,72151,7440.0,"SAN JUAN-BAYAMON, PR",41980.0,"San Juan-Caguas-Guaynabo, PR"
3291,YAUCO,PR,72153,6360.0,"PONCE, PR",49500.0,"Yauco, PR"


In [4]:
abbr2Fips = {
    'AL':1, 'AK':2, 'AZ':4, 'AR':5, 'CA':6, 'CO':8, 'CT':9, 'DE':10, 'FL':12, 'GA':13, 'HI':15, 'ID':16, 'IL':17, 'IN':18, 'IA':19, 'KS':20, 'KY':21, 'LA':22, 'ME':23, 'MD':24, 'MA':25, 'MI':26, 'MN':27, 'MS':28, 'MO':29, 'MT':30, 'NE':31, 'NV':32, 'NH':33, 'NJ':34, 'NM':35, 'NY':36, 'NC':37, 'ND':38, 'OH':39, 'OK':40, 'OR':41, 'PA':42, 'RI':44, 'SC':45, 'SD':46, 'TN':47, 'TX':48, 'UT':49, 'VT':50, 'VA':51, 'WA':53, 'WV':54, 'WI':55, 'WY':56, 'AS':60, 'GU':66, 'MP':69, 'PR':72, 'VI':78, 'DC':0
}

In [5]:
year = 1990
# file_name = 'county_time_%s.csv' % item
file_name = 'IRS_Migration_Matrix_%d.csv' % year
file_path = './migration_v2/%s' % file_name
df = pd.read_csv(file_path, dtype={'Unnamed: 0': str})
df= df.rename(columns={'Unnamed: 0':'fipscounty'})
df.shape

(3149, 3150)

In [6]:
df.head()

,fipscounty,01001,01003,01005,01007,01009,01011,01013,01015,01017,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
0,01001,26703,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01003,0,74857,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01005,0,0,18237,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01007,0,0,0,13373,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01009,0,0,0,0,26910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dfx = df.merge(df_counties, on='fipscounty', how='left')
assert dfx.shape[0] == df.shape[0]
dfx.shape

(3149, 3156)

In [8]:
dfx.head()

,fipscounty,01001,01003,01005,01007,01009,01011,01013,01015,01017,...,56039,56041,56043,56045,countyname,state,msa,msaname,cbsa,cbsaname
0,01001,26703,0,0,0,0,0,0,0,0,...,0,0,0,0,AUTAUGA,AL,5240.0,"MONTGOMERY, AL",33860.0,"Montgomery, AL"
1,01003,0,74857,0,0,0,0,0,0,0,...,0,0,0,0,BALDWIN,AL,5160.0,"MOBILE, AL",NaN,NaN
2,01005,0,0,18237,0,0,0,0,0,0,...,0,0,0,0,BARBOUR,AL,1.0,ALABAMA,NaN,NaN
3,01007,0,0,0,13373,0,0,0,0,0,...,0,0,0,0,BIBB,AL,1.0,ALABAMA,13820.0,"Birmingham-Hoover, AL"
4,01009,0,0,0,0,26910,0,0,0,0,...,0,0,0,0,BLOUNT,AL,1000.0,"BIRMINGHAM, AL",13820.0,"Birmingham-Hoover, AL"


In [9]:
len(dfx['state'].unique())

51

In [10]:
all_states = list(dfx['state'].unique())
all_states_info = []
for state_name in all_states:
    all_states_info.append((state_name, abbr2Fips[state_name], list(dfx.index[dfx['state'] == state_name])))


In [11]:
with open('./migration_v2/all_state_index.txt', 'wt') as output_f:
    for state in all_states_info:
        output_f.write('%s %s %s\n' % (str(state[0]), str(state[1]), ','.join([str(item+1) for item in state[2]])))